# Assignment 2 - Problem A (More Concise, without reading data from external source)


$$ \sum_{i=H,C,M,P,j=H,C,M,P,} r_{ij}x_{ij}\\
s.t. \\
Capacity(HC): x_{HC}+x_{MC}+x_{PC} \le 240 \\
Capacity(HM): x_{HM}+x_{CM}+x_{PM} \le 240 \\
Capacity(HP): x_{HP}+x_{CP}+x_{MP} \le 240 \\
Capacity(CH): x_{CH}+x_{CM}+x_{CP} \le 240 \\
Capacity(MH): x_{MH}+x_{MC}+x_{MP} \le 240 \\
Capacity(PH): x_{PH}+x_{PC}+x_{PM} \le 240 \\
Demand: x_{ij} \le d_{ij}\\
$$

## Step 1: Import PuLP modeler functions

In [1]:
from pulp import *
#import pandas as pd
#import numpy as np

## Step 2: Define a variable using LpProblem to represent the optimization problem

- give the variable a name, for examle **probA**
- use **LpProblem** class: **LpProblem(name='NoName', sense=1)**. Parameters are explained below:
- **name**:name of the problem used in the output .lp file
- **sense**: type of the LP problem objective. Either LpMinimize (default) or LpMaximize.
- LpProblem returns an LP problem

In [2]:
probA=LpProblem("Problem A",LpMaximize)

## Step 3: Define decision variables using LpVariable
- give each decision variable a name, say **xhc** represents the number passengers to fly from Houston to Chicago
- use **LpVariable** class: **LpVariable(name, lowBound=None, upBound=None, cat='Continuous', e=None)**. Parameters are explained below:
- **name**:The name of the variable used in the output .lp file
- **lowBound**: The lower bound on this variable’s range. Default is negative infinity
- **upBound**: The upper bound on this variable’s range. Default is positive infinity
- **cat**:  The category this variable is in, Integer, Binary or Continuous(default)
- **e**: Used for column based modelling: relates to the variable’s existence in the objective function and constraints

In [3]:
# creates a list of all cities
City=['H','C','M','P']
# Creates a list of tuples containing all the possible routes for transport
Routes = [(fr, to) for fr in City for to in City]
print(Routes)
# Creates a list of demand for each route
MaxDemand= [[0,123,80,110],\
            [130, 0, 98, 88],\
            [72, 105, 0, 68],\
            [115,90,66,0]]
# The demand data is made into a dictionary
MaxDemand= makeDict([City,City],MaxDemand)

# Creates a list of fares
Fares = [#'Houston':\
    [0,197,110,125],\
    #'Chicago':
    [190, 0, 282, 195],\
    #'Miami':
    [108, 292, 0, 238],\
    #'Phoenix':
    [110,192,230,0]]
# The fares data is made intot a dictionary
Fares = makeDict([City,City],Fares,0)
Capacity = 240
passenger_vars = LpVariable.dicts("x", (City, City),lowBound=0, cat='Continuous')

[('H', 'H'), ('H', 'C'), ('H', 'M'), ('H', 'P'), ('C', 'H'), ('C', 'C'), ('C', 'M'), ('C', 'P'), ('M', 'H'), ('M', 'C'), ('M', 'M'), ('M', 'P'), ('P', 'H'), ('P', 'C'), ('P', 'M'), ('P', 'P')]


In [4]:
#objective function
probA+=lpSum([passenger_vars[fr][to]*Fares[fr][to] for (fr,to) in Routes])
# outbound capacity constraint
for i in City[1:]:
    probA += lpSum([passenger_vars[i][j] for j in City ]) <= Capacity,"outbound_%s"%i

# inbound capacity constraint    
for j in City[1:]:
    probA += lpSum([passenger_vars[i][j] for i in City ]) <= Capacity,"inbound_%s"%j

# demand constraint
for i in City:
    for j in City:
        probA += passenger_vars[i][j] <= MaxDemand[i][j],"D%s%s"%(i,j)

## Step 5: Run solver

- use name.solve(solver=None), where name is the LP problem variable defined by LpProblem
- Solve the given Lp problem. 
- This function changes the problem to make it suitable for solving then calls the solver.actualSolve method to find the solution. 
- solver – Optional: the specific solver to be used, defaults to the default solver.

In [5]:
probA.writeLP("Bluesky2.lp")
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


## Step 6: Print the optiomal solution

In [8]:
for v in probA.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total revenue=", value(probA.objective))

x_C_C = 0.0 	Reduced Cost = -382.0
x_C_H = 84.0 	Reduced Cost = 0.0
x_C_M = 94.0 	Reduced Cost = 0.0
x_C_P = 62.0 	Reduced Cost = 0.0
x_H_C = 123.0 	Reduced Cost = 0.0
x_H_H = 0.0 	Reduced Cost = 0.0
x_H_M = 80.0 	Reduced Cost = 0.0
x_H_P = 110.0 	Reduced Cost = 0.0
x_M_C = 100.0 	Reduced Cost = 0.0
x_M_H = 72.0 	Reduced Cost = 0.0
x_M_M = 0.0 	Reduced Cost = -192.0
x_M_P = 68.0 	Reduced Cost = 0.0
x_P_C = 17.0 	Reduced Cost = 0.0
x_P_H = 115.0 	Reduced Cost = 0.0
x_P_M = 66.0 	Reduced Cost = 0.0
x_P_P = 0.0 	Reduced Cost = -5.0
Total revenue= 185593.0


In [7]:
print("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
for name, c in list(probA.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)



Sensitivity Analysis
Constraint		Shadow Price	Slack
outbound_C : x_C_C + x_C_H + x_C_M + x_C_P <= 240 	 190.0 		 -0.0
outbound_M : x_M_C + x_M_H + x_M_M + x_M_P <= 240 	 100.0 		 -0.0
outbound_P : x_P_C + x_P_H + x_P_M + x_P_P <= 240 	 -0.0 		 42.0
inbound_C : x_C_C + x_H_C + x_M_C + x_P_C <= 240 	 192.0 		 -0.0
inbound_M : x_C_M + x_H_M + x_M_M + x_P_M <= 240 	 92.0 		 -0.0
inbound_P : x_C_P + x_H_P + x_M_P + x_P_P <= 240 	 5.0 		 -0.0
DHH : x_H_H <= 0 	 -0.0 		 -0.0
DHC : x_H_C <= 123 	 5.0 		 -0.0
DHM : x_H_M <= 80 	 18.0 		 -0.0
DHP : x_H_P <= 110 	 120.0 		 -0.0
DCH : x_C_H <= 130 	 -0.0 		 46.0
DCC : x_C_C <= 0 	 -0.0 		 -0.0
DCM : x_C_M <= 98 	 -0.0 		 4.0
DCP : x_C_P <= 88 	 -0.0 		 26.0
DMH : x_M_H <= 72 	 8.0 		 -0.0
DMC : x_M_C <= 105 	 -0.0 		 5.0
DMM : x_M_M <= 0 	 -0.0 		 -0.0
DMP : x_M_P <= 68 	 133.0 		 -0.0
DPH : x_P_H <= 115 	 110.0 		 -0.0
DPC : x_P_C <= 90 	 -0.0 		 73.0
DPM : x_P_M <= 66 	 138.0 		 -0.0
DPP : x_P_P <= 0 	 -0.0 		 -0.0


## Problem C


$$ \sum_{i=H,C,M,P,j=H,C,M,P,} r_{ij}x_{ij} -\sum_{i=C,M,P}(24000+80y_i)\\
s.t. \\
Capacity(HC): x_{HC}+x_{MC}+x_{PC} -y_C \le 240 \\
Capacity(HM): x_{HM}+x_{CM}+x_{PM} -y_M \le 240 \\
Capacity(HP): x_{HP}+x_{CP}+x_{MP} -y_P \le 240 \\
Capacity(CH): x_{CH}+x_{CM}+x_{CP} -y_C\le 240 \\
Capacity(MH): x_{MH}+x_{MC}+x_{MP} -y_H\le 240 \\
Capacity(PH): x_{PH}+x_{PC}+x_{PM} -y_P\le 240 \\
Demand: x_{ij} \le d_{ij}\\
$$

In [75]:
probC=LpProblem("Problem C",LpMaximize)
y_vars=LpVariable.dict("y_%s",["C","M","P"],lowBound=0)
probC+=lpSum([passenger_vars[fr][to]*Fares[fr][to] for (fr,to) in Routes])-lpSum([24000+80*y_vars[i] for i in ["C","M","P"]]), "profit"
for i in City[1:]:
    probC += lpSum([passenger_vars[i][j] for j in City])-y_vars[i] <= 0,"outbound_%s"%i

# inbound capacity constraint    
for j in City[1:]:
    probC += lpSum([passenger_vars[i][j] for i in City])-y_vars[j] <= 0,"inbound_%s"%j

# demand constraint
for i in City:
    for j in City:
        probC += passenger_vars[i][j] <= MaxDemand[i][j],"D%s%s"%(i,j)
statusC=probC.solve()
probC.writeLP("Bluesky2C.lp")
print("Status:",LpStatus[probC.status])

Status: Optimal


In [76]:
for v in probC.variables():
    print(v.name, "=", v.varValue,"\tReduced Cost =", v.dj)
print("Total revenue=", value(probC.objective))

x_C_C = 0.0 	Reduced Cost = -80.0
x_C_H = 130.0 	Reduced Cost = 0.0
x_C_M = 98.0 	Reduced Cost = 0.0
x_C_P = 88.0 	Reduced Cost = 0.0
x_H_C = 123.0 	Reduced Cost = 0.0
x_H_H = 0.0 	Reduced Cost = 0.0
x_H_M = 80.0 	Reduced Cost = 0.0
x_H_P = 110.0 	Reduced Cost = 0.0
x_M_C = 105.0 	Reduced Cost = 0.0
x_M_H = 72.0 	Reduced Cost = 0.0
x_M_M = 0.0 	Reduced Cost = -80.0
x_M_P = 68.0 	Reduced Cost = 0.0
x_P_C = 90.0 	Reduced Cost = 0.0
x_P_H = 115.0 	Reduced Cost = 0.0
x_P_M = 66.0 	Reduced Cost = 0.0
x_P_P = 0.0 	Reduced Cost = -80.0
y_C = 318.0 	Reduced Cost = 0.0
y_M = 245.0 	Reduced Cost = 0.0
y_P = 271.0 	Reduced Cost = 0.0
Total revenue= 77287.0


In [77]:
print("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
for name, c in list(probA.constraints.items()):
    print(name, ":", c, "\t", c.pi, "\t\t", c.slack)



Sensitivity Analysis
Constraint		Shadow Price	Slack
outbound_C : x_C_C + x_C_H + x_C_M + x_C_P <= 240 	 190.0 		 -0.0
outbound_M : x_M_C + x_M_H + x_M_M + x_M_P <= 240 	 100.0 		 -0.0
outbound_P : x_P_C + x_P_H + x_P_M + x_P_P <= 240 	 -0.0 		 42.0
inbound_C : x_C_C + x_H_C + x_M_C + x_P_C <= 240 	 192.0 		 -0.0
inbound_M : x_C_M + x_H_M + x_M_M + x_P_M <= 240 	 92.0 		 -0.0
inbound_P : x_C_P + x_H_P + x_M_P + x_P_P <= 240 	 5.0 		 -0.0
DHH : x_H_H <= 0 	 -0.0 		 -0.0
DHC : x_H_C <= 123 	 5.0 		 -0.0
DHM : x_H_M <= 80 	 18.0 		 -0.0
DHP : x_H_P <= 110 	 120.0 		 -0.0
DCH : x_C_H <= 130 	 -0.0 		 46.0
DCC : x_C_C <= 0 	 -0.0 		 -0.0
DCM : x_C_M <= 98 	 -0.0 		 4.0
DCP : x_C_P <= 88 	 -0.0 		 26.0
DMH : x_M_H <= 72 	 8.0 		 -0.0
DMC : x_M_C <= 105 	 -0.0 		 5.0
DMM : x_M_M <= 0 	 -0.0 		 -0.0
DMP : x_M_P <= 68 	 133.0 		 -0.0
DPH : x_P_H <= 115 	 110.0 		 -0.0
DPC : x_P_C <= 90 	 -0.0 		 73.0
DPM : x_P_M <= 66 	 138.0 		 -0.0
DPP : x_P_P <= 0 	 -0.0 		 -0.0
